In [1]:
#imports
import pickle
import numpy as np
import glob
import os
import pandas as pd

In [2]:
#load infant dataset
with open("./Data/new_InfAct_plus_2d.pkl", "rb") as f:
    ds = pickle.load(f)

In [3]:
num_acts_per_sub = {}
for anno in ds['annotations']:
    source = anno['source']
    pos = anno['pos_label']
    if source not in num_acts_per_sub.keys():
        num_acts_per_sub[source] = {}
    if pos not in num_acts_per_sub[source].keys():
        num_acts_per_sub[source][pos] = 1
    else:
        num_acts_per_sub[source][pos]+=1

In [4]:
num_acts_per_sub_dict = pd.DataFrame.from_dict(num_acts_per_sub)
num_acts_per_sub_dict.fillna(0, inplace=True)
num_acts_per_sub_dict

,youtube,CAREER_D01,CAREER_D05,CAREER_D08,CAREER_D09,CAREER_D02,CAREER_D03,CAREER_D10
Prone,74,75.0,45.0,30.0,28.0,1,6.0,44.0
All-fours,69,0.0,0.0,0.0,0.0,22,70.0,81.0
Standing,79,0.0,0.0,0.0,0.0,21,37.0,0.0
Supine,62,23.0,0.0,0.0,45.0,1,0.0,18.0
Sitting,116,0.0,0.0,0.0,0.0,33,138.0,101.0


In [5]:
get_source = lambda x: x['source']
get_frame_dirs = lambda x: x['frame_dir']

In [6]:
sources = list(map(get_source, ds['annotations']))
u_sources = np.unique(sources)

for us in u_sources:
    non_youtube_ann = [item for item in ds['annotations'] if "youtube" not in item['source']]
    if us=="youtube":
        continue
    leave_out_sub = us

    yt_splits    = [item for item in ds["annotations"] if "youtube" in item['source']]
    train_splits = [item for item in non_youtube_ann if leave_out_sub not in item['source']]
    val_splits   = [item for item in non_youtube_ann if leave_out_sub in item['source']]
    assert len(yt_splits)==400
    assert len(train_splits)+len(val_splits)==819

    leave_out_ds = {"split": {
        "train": [*list(map(get_frame_dirs, train_splits))],
        "val": [*list(map(get_frame_dirs, val_splits))]
    },
    "annotations": [*train_splits, *val_splits]}
    assert len(leave_out_ds['annotations'])==819
    assert len(leave_out_ds['split']['train'])+len(leave_out_ds['split']['val'])==len(leave_out_ds['annotations'])

    for item in leave_out_ds['split']['train']:
        assert item not in leave_out_ds['split']['val']

        
    for item in leave_out_ds['split']['val']:
        assert item not in leave_out_ds['split']['train']

    leave_out_ds_with_yt = {"split": {
        "train": [*list(map(get_frame_dirs, train_splits)), *list(map(get_frame_dirs, yt_splits))],
        "val": [*list(map(get_frame_dirs, val_splits))]
    },
    "annotations": [*train_splits, *val_splits, *yt_splits]}
    assert len(leave_out_ds_with_yt['annotations'])==1219
    assert len(leave_out_ds_with_yt['split']['train'])+len(leave_out_ds_with_yt['split']['val'])==len(leave_out_ds_with_yt['annotations'])

    with open(f"Data\leave_out\leave_out_{leave_out_sub}.pkl", "wb") as f:
        pickle.dump(leave_out_ds, f)

    with open(f"Data\leave_out\leave_out_{leave_out_sub}_with_youtube.pkl", "wb") as f:
        pickle.dump(leave_out_ds_with_yt, f)

In [7]:
#mim train mmaction configs_career\ctrgcn_infact_plus_2d_primitive_leave_out_D01.py   --work-dir ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D01_run1
#mim train mmaction configs_career\ctrgcn_infact_plus_2d_primitive_leave_out_D02.py   --work-dir ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D02_run1
#mim train mmaction configs_career\ctrgcn_infact_plus_2d_primitive_leave_out_D03.py   --work-dir ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D03_run1
#mim train mmaction configs_career\ctrgcn_infact_plus_2d_primitive_leave_out_D05.py   --work-dir ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D05_run1
#mim train mmaction configs_career\ctrgcn_infact_plus_2d_primitive_leave_out_D08.py   --work-dir ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D08_run1
#mim train mmaction configs_career\ctrgcn_infact_plus_2d_primitive_leave_out_D09.py   --work-dir ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D09_run1
#mim train mmaction configs_career\ctrgcn_infact_plus_2d_primitive_leave_out_D10.py   --work-dir ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D10_run1

In [50]:
#mim test mmaction configs_career/ctrgcn_infact_plus_2d_primitive_leave_out_D01.py --checkpoint ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D01_run1/epoch_50.pth --dump ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D01_run1/eval.pkl
#mim test mmaction configs_career/ctrgcn_infact_plus_2d_primitive_leave_out_D02.py --checkpoint ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D02_run1/epoch_50.pth --dump ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D02_run1/eval.pkl
#mim test mmaction configs_career/ctrgcn_infact_plus_2d_primitive_leave_out_D03.py --checkpoint ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D03_run1/epoch_50.pth --dump ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D03_run1/eval.pkl
#mim test mmaction configs_career/ctrgcn_infact_plus_2d_primitive_leave_out_D05.py --checkpoint ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D05_run1/epoch_50.pth --dump ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D05_run1/eval.pkl
#mim test mmaction configs_career/ctrgcn_infact_plus_2d_primitive_leave_out_D08.py --checkpoint ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D08_run1/epoch_50.pth --dump ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D08_run1/eval.pkl
#mim test mmaction configs_career/ctrgcn_infact_plus_2d_primitive_leave_out_D09.py --checkpoint ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D09_run1/epoch_50.pth --dump ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D09_run1/eval.pkl
#mim test mmaction configs_career/ctrgcn_infact_plus_2d_primitive_leave_out_D10.py --checkpoint ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D10_run1/epoch_50.pth --dump ../../../Results_CTRGCN/ctrgcn_infact_plus_2d_primitive_leave_out_D10_run1/eval.pkl

In [61]:
convert_to_percent = lambda x: round(x*100, 2)

model = "STGCN"

results_pths = glob.glob(f"Results_{model}\\*_leave_out_*\\eval.pkl")
results_dicts = {}
for res_path in results_pths:
    pth_split = res_path.split(os.sep)
    exp_num = f"{model} Exp: "+pth_split[-2].split("_")[-1]
    leave_out_sub = pth_split[1].split("_")[-2]
    with open(res_path, "rb") as f:
        eval = pickle.load(f)
    gts = np.array([item['gt_label'].item() for item in eval])
    preds = np.array([item['pred_label'].item() for item in eval])
    acc = (gts==preds).sum()/len(gts)
    acc = convert_to_percent(acc)
    
    if leave_out_sub not in results_dicts.keys():
        results_dicts[leave_out_sub] = {}
    results_dicts[leave_out_sub][exp_num]=acc

results_df = pd.DataFrame.from_dict(results_dicts)
results_df['Subject Mean'] = results_df.mean(axis=1)
exp_means = results_df.mean(axis=0)
results_df.loc[f'{model} Exp means'] = exp_means

results_df

,D01,D02,D03,D05,D08,D09,Subject Mean
STGCN Exp: run1,76.53,60.260000,53.780000,35.560000,100.0,75.340000,66.911667
STGCN Exp: run2,76.53,56.410000,51.390000,20.000000,100.0,75.340000,63.278333
STGCN Exp: run3,76.53,58.970000,51.000000,51.110000,100.0,76.710000,69.053333
STGCN Exp means,76.53,58.546667,52.056667,35.556667,100.0,75.796667,66.414444


In [52]:
#python tools/train.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D01.py --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D01
#python tools/train.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D02.py --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D02
#python tools/train.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D03.py --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D03
#python tools/train.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D05.py --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D05
#python tools/train.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D08.py --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D08
#python tools/train.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D09.py --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D09

In [53]:
#python tools\test.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D01.py ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D01\best_acc_top1_epoch_1.pth --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D01 --dump ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D01\eval.pkl
#python tools\test.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D02.py ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D02\best_acc_top1_epoch_37.pth --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D02 --dump ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D02\eval.pkl
#python tools\test.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D03.py ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D03\best_acc_top1_epoch_17.pth --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D03 --dump ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D03\eval.pkl
#python tools\test.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D05.py ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D05\best_acc_top1_epoch_35.pth --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D05 --dump ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D05\eval.pkl
#python tools\test.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D08.py ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D08\best_acc_top1_epoch_6.pth --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D08 --dump ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D08\eval.pkl
#python tools\test.py configs\skeleton\career_stgcn\stgcn_infact_plus_2d_primitive_leave_out_D09.py ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D09\best_acc_top1_epoch_14.pth --work-dir ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D09 --dump ..\Results_STGCN_CAREER\stgcn_infact_plus_2d_primitive_leave_out_D09\eval.pkl

In [56]:
convert_to_percent = lambda x: round(x*100, 2)

results_pths = glob.glob("Results_STGCN_CAREER_*\\*\\eval.pkl")
results_dicts = {}
for res_path in results_pths:
    pth_split = res_path.split(os.sep)
    exp_num = "STGCN Exp: "+pth_split[0].split("_")[-1]
    leave_out_sub = pth_split[1].split("_")[-1]
    with open(res_path, "rb") as f:
        eval = pickle.load(f)
    gts = np.array([item['gt_label'].item() for item in eval])
    preds = np.array([item['pred_label'].item() for item in eval])
    acc = (gts==preds).sum()/len(gts)
    acc = convert_to_percent(acc)
    
    if leave_out_sub not in results_dicts.keys():
        results_dicts[leave_out_sub] = {}
    results_dicts[leave_out_sub][exp_num]=acc

results_df = pd.DataFrame.from_dict(results_dicts)
results_df['Subject Mean'] = results_df.mean(axis=1)
exp_means = results_df.mean(axis=0)
results_df.loc['STGCN Exp means'] = exp_means

,D01,D02,D03,D05,D08,D09,Subject Mean
CTRGCN Exp: 1,76.53,60.260000,53.780000,35.560000,100.0,75.340000,66.911667
CTRGCN Exp: 2,76.53,56.410000,51.390000,20.000000,100.0,75.340000,63.278333
CTRGCN Exp: 3,76.53,58.970000,51.000000,51.110000,100.0,76.710000,69.053333
STGCN Exp means,76.53,58.546667,52.056667,35.556667,100.0,75.796667,66.414444
